In [45]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns

### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

In [46]:
%jsroot on

In [47]:
def getListOfFiles(dirName, histos=False):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    #To ensure event counts don't come up against  a histogram file and rather just the nanoskim
    if histos==False: listOfFile[:] = [x for x in listOfFile if "histograms" not in x]
    #print (listOfFile)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):            
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

print (getListOfFiles('/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/'))

['/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/jetObservables_nanoskim_30.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/jetObservables_nanoskim_31.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/jetObservables_nanoskim_44.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/jetObservables_nanoskim_32.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Sk

In [48]:
nano_directories = [
    #'/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTTo2L2Nu_MiniAODv2_v1/210501_145150/0000/',

    #'/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/', 
    #'/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0001/', 

    #'/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/jetObservables_Skimmer_UL17v2_TTJets_MiniAODv2_V2/210502_173402/0000/', 
    #'/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/jetObservables_Skimmer_UL17v2_TTJets_MiniAODv2_V2/210502_173402/0001/', 

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_s-channel_4f_leptonDecays_TuneCP5_13TeV-amcatnlo-pythia8/jetObservables_Skimmer_UL17v2_ST_s-channel_MiniAODv2_V2/210502_175243/0000/',

    #'/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_top_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17_ST_t-channel_top_new_msoftdrop/210419_090045/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_antitop_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_ST_t-channel_antitop_MiniAODv2_V2/210502_173528/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_ST_tW_top_MiniAODv2_V2/210502_173846/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_ST_tW_antitop_MiniAODv2_V2/210502_173634/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/jetObservables_Skimmer_UL17v2_WJetsToLNu_MiniAODv2_v1/210501_151753/0000/', 

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-170to300_MuEnrichedPt5_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17v2_QCDMuEn_Pt170to300_MiniAODv2_v1/210501_151046/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-300to470_MuEnrichedPt5_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17v2_QCDMuEn_Pt300to470_MiniAODv2_V2/210502_173952/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-470to600_MuEnrichedPt5_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17v2_QCDMuEn_Pt470to600_MiniAODv2_v1/210501_150434/0000/',  

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-600to800_MuEnrichedPt5_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17v2_QCDMuEn_Pt600to800_MiniAODv2_V2/210502_175453/0000/',  

    #'/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt800to1000_new_msoftdrop/210416_001758/0000/',  

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-1000toInf_MuEnrichedPt5_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17v2_QCDMuEn_Pt1000toInf_MiniAODv2_v1/210501_150706/0000',  

    ]

In [49]:
dict_MCscaling_UL17 = {
    'TTJets': ["722.8", 285095726103.23967,
               '../TTJets_amcatnloFXFX-pythia8_2017UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 32392787477.908009,
                         '../TTToSemileptonic_powheg_pythia8_2017UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 4759777104.469412, 
                  '../TTTo2L2Nu_powheg_pythia8_2017UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 8110137121728.131836, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 70439807.475972, 
                     '../ST_s-channel_amcatnlo_pythia8_2017UL.root'
                    ],
    #'ST_t-channel_top': ["1.197e+02", 655197632.514000, 
    #                     '../ST_t-channel_top_powheg_pythia8_2017UL.root'
    #                    ],
    'ST_t-channel_antitop': ["7.174e+01", 265124234.844000, 
                            '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 331239851.967700, 
                  '../ST_tW_top_powheg_pythia8_2017UL.root'
                 ],
    'ST_tW_antitop': ["3.251e+01", 303960792.435600, 
                     '../ST_tW_antitop_powheg_pythia8_2017UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Pt_170to300_pythia8_2017UL.root'
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Pt_300to470_pythia8_2017UL.root'   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Pt_470to600_pythia8_2017UL.root'
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Pt_600to800_pythia8_2017UL.root'
                       ],
    #'QCD_Pt_800to1000': ["2.597e+01", 39261600.000000, 
    #                     '../QCD_Pt_800to1000_pythia8_2017UL.root'    
    #                    ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Pt_1000to1400_pythia8_2017UL.root'
                         ],
    
    #'WW': ["7.577e+01", 7876265.259367, 
    #      '../WW_pythia8_2017UL.root'  
    #      ],
    #'WZ': ["1.21e+00", 3970000.000000, 
    #       '../WZ_pythia8_2017UL.root'    
    #      ],
    #'ZZ': ["2.748e+00", 1981800.000000, 
    #       '../ZZ_pythia8_2017UL.root'
    #      ]
    }


In [50]:
lumi=4653.71+9289.53+11758.83
print (lumi)

samples = list(dict_MCscaling_UL17.keys())
dict_items = list(dict_MCscaling_UL17.items())
xsecs = np.array(list(float(dict_items[k][1][0]) for k in np.arange(0,len(dict_items)))) 
nevts = np.array(list(float(dict_items[k][1][1]) for k in np.arange(0,len(dict_items)))) 
fpathsMC = np.array(list((dict_items[k][1][2]) for k in np.arange(0,len(dict_items))))
fpathData = '../SingleMuonAll_2017UL.root'
print (xsecs, samples, nevts, fpathsMC)

25702.07
[7.2280e+02 3.6534e+02 8.8290e+01 5.3680e+04 3.5490e+00 7.1740e+01
 3.2450e+01 3.2510e+01 7.0550e+03 6.1930e+02 5.9240e+01 1.8210e+01
 1.0780e+00] ['TTJets', 'TTToSemiLeptonic', 'TTTo2L2Nu', 'WJetsToLNu', 'ST_s-channel', 'ST_t-channel_antitop', 'ST_tW_top', 'ST_tW_antitop', 'QCD_Pt_170to300', 'QCD_Pt_300to470', 'QCD_Pt_470to600', 'QCD_Pt_600to800', 'QCD_Pt_1000toInf'] [2.85095726e+11 3.23927875e+10 4.75977710e+09 8.11013712e+12
 7.04398075e+07 2.65124235e+08 3.31239852e+08 3.03960792e+08
 3.60276730e+07 4.94796073e+05 5.17383714e+05 1.73188126e+07
 1.46425530e+07] ['../TTJets_amcatnloFXFX-pythia8_2017UL.root'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root'
 '../TTTo2L2Nu_powheg_pythia8_2017UL.root'
 '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root'
 '../ST_s-channel_amcatnlo_pythia8_2017UL.root'
 '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
 '../ST_tW_top_powheg_pythia8_2017UL.root'
 '../ST_tW_antitop_powheg_pythia8_2017UL.root'
 '../QCD_Pt_170to300_pythia8_2017UL.root

In [51]:
print (len((dict_MCscaling_UL17.keys())))

13


In [52]:
import array
from array import array

In [53]:
def Bin(Range):
    bins=np.array([x for x in np.linspace(num=Range[0]+1,start=Range[1],stop=Range[2])])
    return bins

def getRebinnedRescaledHist(hist_name, file_name, xrange, lumi, xsec, nevt, isMC=True, rebin=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    if rebin: newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    else: newHist = hist.Clone()
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(lumi*xsec/nevt)
    
    return newHist

In [54]:
Hists_topSel=np.array([
    'recoJet_pt_nom_topSel', 'recoJet_mass_nom_topSel', #'recoJetTau21_nom_topSel', 'recoJetTau32_nom_topSel',
    'recoJet_tau_0p5_1_nom_topSel', 'recoJet_tau_1_1_nom_topSel', 'recoJet_tau_2_1_nom_topSel',
    'recoJet_tau_0p5_2_nom_topSel', 'recoJet_tau_1_2_nom_topSel', 'recoJet_tau_2_2_nom_topSel',
    'recoJet_tau_0p5_3_nom_topSel', 'recoJet_tau_1_3_nom_topSel', 'recoJet_tau_2_3_nom_topSel',
    'recoJet_tau_0p5_4_nom_topSel', 'recoJet_tau_1_4_nom_topSel', 'recoJet_tau_2_4_nom_topSel',
    #'resolJet_tau_0p5_1_topSel', 'resolJet_tau_1_1_topSel', 'resolJet_tau_2_1_topSel',
    #'resolJet_tau_0p5_2_topSel', 'resolJet_tau_1_2_topSel', 'resolJet_tau_2_2_topSel',
    #'resolJet_tau_0p5_3_topSel', 'resolJet_tau_1_3_topSel', 'resolJet_tau_2_3_topSel',
    #'resolJet_tau_0p5_4_topSel', 'resolJet_tau_1_4_topSel', 'resolJet_tau_2_4_topSel',
    'nPVs_topSel','nleps_topSel',
    'muons_pt_topSel','muons_eta_topSel','muons_phi_topSel',
    'nAK8jets_topSel','AK8jets_pt_topSel','AK8jets_eta_topSel','AK8jets_phi_topSel','AK8jets_mass_topSel',
    'nAK4jets_topSel','AK4jets_pt_topSel','AK4jets_eta_topSel','AK4jets_phi_topSel','AK4jets_mass_topSel',
    'METPt_topSel','HT_topSel'])
#Objects_topSel=np.concatenate((Objects1_topSel,Objects3_topSel),axis=0)

Histnames_topSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],#['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    #['Events/0.03','#resol tau_{1}^{(0.5)}','resol #tau_{1}^{(0.5)}'],['Events/0.02','resol #tau_{1}^{(1)}','resol #tau_{1}^{(1)}'],['Events/0.02','resol #tau_{1}^{(2)}','resol #tau_{1}^{(2)}'],
    #['Events/0.03','#resol tau_{2}^{(0.5)}','resol #tau_{2}^{(0.5)}'],['Events/0.02','resol #tau_{2}^{(1)}','resol #tau_{2}^{(1)}'],['Events/0.01','resol #tau_{2}^{(2)}','resol #tau_{2}^{(2)}'],
    #['Events/0.02','#resol tau_{3}^{(0.5)}','resol #tau_{3}^{(0.5)}'],['Events/0.02','resol #tau_{3}^{(1)}','resol #tau_{3}^{(1)}'],['Events/0.01','resol #tau_{3}^{(2)}','resol #tau_{3}^{(2)}'],
    #['Events/0.02','#resol tau_{4}^{(0.5)}','resol #tau_{4}^{(0.5)}'],['Events/0.01','resol #tau_{4}^{(1)}','resol #tau_{4}^{(1)}'],['Events/0.01','resol #tau_{4}^{(2)}','resol #tau_{4}^{(2)}'],
    ['Events/2','Number of PV','Number of PV'],['Events','Number of Leptons','Number of Leptons'],
    ['Events/(20 GeV)','P_{t} of Muons','P_{t} of Muons/GeV'],['Events/0.4','#eta of Muons','#eta of Muons'],['Events/(#pi/10)','#phi of Muons','#phi of Muons'],
    ['Events','Number of AK8Jets','Number of AK8Jets'],['Events/(60 GeV)','P_{t} of AK8Jets','P_{t} of AK8Jets/GeV'],['Events/0.4','#eta of AK8Jets','#eta of AK8Jets'],['Events/(#pi/10)','#phi of AK8Jets','#phi of AK8Jets'],['Events/(10 GeV)','Mass of AK8Jets','Mass of AK8Jets/GeV'],
    ['Events','Number of AK4Jets','Number of AK4Jets'],['Events/(40 GeV)','P_{t} of AK4Jets','P_{t} of AK4Jets/GeV'],['Events/0.4','#eta of AK4Jets','#eta of AK4Jets'],['Events/(#pi/10)','#phi of AK4Jets','#phi of AK4Jets'],['Events/(20 GeV)','Mass of AK4Jets','Mass of AK4Jets/GeV'],
    ['Events/(20 GeV)','MET P_{t}','MET P_{t}/GeV'],['Events/(100 GeV)','HT','HT/GeV']])
Range_topSel=[
    [30,0,1500], [12,130,250],#[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1],
    
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    
    [35,0,70],[4,0,4],
    [20,0,400],[20,-4,4],[20,-3.14259, 3.14159],
    [8,0,8],[20,0,1200],[20,-4,4],[20,-3.14259, 3.14159],[15,0,150],
    [8,0,8],[20,0,800],[20,-4,4],[20,-3.14259, 3.14159],[20,0,400],
    [30,0,600],[20,0,2000]
]
print (len(Hists_topSel),len(Histnames_topSel),len(Range_topSel))

31 31 31


In [55]:
print (Histnames_topSel[0][0]
      )

Events/(50 GeV)


In [56]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [57]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_topSel[i])
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillColor(ROOT.kGreen+3)
    
    TTDileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[2], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[2], nevt=nevts[2]))
    TTDileptonic.SetLineColor(ROOT.kRed-9)
    TTDileptonic.SetFillColor(ROOT.kRed-9)
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_topSel[i])
    for j in range(4,8):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    QCD = ROOT.TH1F("",";;",*Range_topSel[i])
    for j in range(8,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(ROOT.kPink-1)
    
    #VV = ROOT.TH1F("",";;",*Range_topSel[i])
    #for j in range(19,22):
    #    hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
    #                                    file_name=fpathsMC[j], xrange=Range_topSel[i], 
    #                                    lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
    #    VV.Add(hist)
    #VV.SetLineColor(ROOT.kGray+2)
    #VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTSemileptonic.GetMaximum())
    #MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTDileptonic)
    MC.Add(TTSemileptonic)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_topSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuonUL17 = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathData, xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuonUL17.SetMarkerSize(1.5)
    Data_SingleMuonUL17.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL17.SetMarkerStyle(20)
    Data_SingleMuonUL17.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL17.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    ratio.GetXaxis().SetTitle(Histnames_topSel[i][2])
    ratio.Add(Data_SingleMuonUL17)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    mc_rt.Add(TTSemileptonic)
    mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    # mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"TTSemileptonic","f")
    l[i].AddEntry(TTDileptonic,"TTDileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    #l[i].AddEntry(VV,"VV","f")
    l[i].AddEntry(Data_SingleMuonUL17,"Data UL 2017","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("topSel/"+Hists_topSel[i]+"_powheg_p8.png")
    c[i].Close()

0 recoJet_pt_nom_topSel ../SingleMuonAll_2017UL.root [30, 0, 1500] 25702.07 365.34 32392787477.90801
P_{t}/GeV
1 recoJet_mass_nom_topSel ../SingleMuonAll_2017UL.root [12, 130, 250] 25702.07 365.34 32392787477.90801
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_topSel ../SingleMuonAll_2017UL.root [30, 0, 0.9] 25702.07 365.34 32392787477.90801
#tau_{1}^{(0.5)}
3 recoJet_tau_1_1_nom_topSel ../SingleMuonAll_2017UL.root [35, 0, 0.7] 25702.07 365.34 32392787477.90801
#tau_{1}^{(1)}
4 recoJet_tau_2_1_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{1}^{(2)}
5 recoJet_tau_0p5_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.6] 25702.07 365.34 32392787477.90801
#tau_{2}^{(0.5)}
6 recoJet_tau_1_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{2}^{(1)}
7 recoJet_tau_2_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.2] 25702.07 365.34 32392787477.90801
#tau_{2}^{(2)}
8 recoJet_tau_0p5_3_nom_topSel ../Singl

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_pt_nom_topSel_powheg_p8.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_mass_nom_topSel_powheg_p8.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_tau_0p5_1_nom_topSel_powheg_p8.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_tau_1_1_nom_topSel_powheg_p8.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_tau_2_1_nom_topSel_powheg_p8.png has been created
Info i

In [58]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_topSel[i])
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0]))
    TTJets.SetLineColor(ROOT.kGreen+3)
    TTJets.SetFillColor(ROOT.kGreen+3)
    

    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_topSel[i])
    for j in range(4,8):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    QCD = ROOT.TH1F("",";;",*Range_topSel[i])
    for j in range(8,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(ROOT.kPink-1)
    
    #VV = ROOT.TH1F("",";;",*Range_topSel[i])
    #for j in range(19,22):
    #    hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
    #                                    file_name=fpathsMC[j], xrange=Range_topSel[i], 
    #                                    lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
    #    VV.Add(hist)
    #VV.SetLineColor(ROOT.kGray+2)
    #VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTJets.GetMaximum())
    #MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    #MC.Add(TTDileptonic)
    MC.Add(TTJets)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTJets.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_topSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTJets.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuonUL17 = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathData, xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuonUL17.SetMarkerSize(1.5)
    Data_SingleMuonUL17.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL17.SetMarkerStyle(20)
    Data_SingleMuonUL17.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL17.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    ratio.GetXaxis().SetTitle(Histnames_topSel[i][2])
    ratio.Add(Data_SingleMuonUL17)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    mc_rt.Add(TTJets)
    #mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    #mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTJets,"TTJets","f")
    #l[i].AddEntry(TTDileptonic,"TTDileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    #l[i].AddEntry(VV,"VV","f")
    l[i].AddEntry(Data_SingleMuonUL17,"Data UL 2017","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("topSel/"+Hists_topSel[i]+"_aMCatNLO.png")
    c[i].Close()

0 recoJet_pt_nom_topSel ../SingleMuonAll_2017UL.root [30, 0, 1500] 25702.07 365.34 32392787477.90801
P_{t}/GeV
1 recoJet_mass_nom_topSel ../SingleMuonAll_2017UL.root [12, 130, 250] 25702.07 365.34 32392787477.90801
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_topSel ../SingleMuonAll_2017UL.root [30, 0, 0.9] 25702.07 365.34 32392787477.90801
#tau_{1}^{(0.5)}
3 recoJet_tau_1_1_nom_topSel ../SingleMuonAll_2017UL.root [35, 0, 0.7] 25702.07 365.34 32392787477.90801
#tau_{1}^{(1)}
4 recoJet_tau_2_1_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{1}^{(2)}
5 recoJet_tau_0p5_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.6] 25702.07 365.34 32392787477.90801
#tau_{2}^{(0.5)}
6 recoJet_tau_1_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{2}^{(1)}
7 recoJet_tau_2_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.2] 25702.07 365.34 32392787477.90801
#tau_{2}^{(2)}
8 recoJet_tau_0p5_3_nom_topSel ../Singl

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_pt_nom_topSel_aMCatNLO.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_mass_nom_topSel_aMCatNLO.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_tau_0p5_1_nom_topSel_aMCatNLO.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_tau_1_1_nom_topSel_aMCatNLO.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file topSel/recoJet_tau_2_1_nom_topSel_aMCatNLO.png has been created
Info in <TH

### Resolution plots


In [59]:
Hists_topSel=np.array([
    
    'resolJet_tau_0p5_1_topSel', 'resolJet_tau_1_1_topSel', 'resolJet_tau_2_1_topSel',
    'resolJet_tau_0p5_2_topSel', 'resolJet_tau_1_2_topSel', 'resolJet_tau_2_2_topSel',
    'resolJet_tau_0p5_3_topSel', 'resolJet_tau_1_3_topSel', 'resolJet_tau_2_3_topSel',
    'resolJet_tau_0p5_4_topSel', 'resolJet_tau_1_4_topSel', 'resolJet_tau_2_4_topSel',
    'resolJet_tau21_topSel', 'resolJet_tau32_topSel',
])
#Objects_topSel=np.concatenate((Objects1_topSel,Objects3_topSel),axis=0)

Histnames_topSel=np.array([
    ['Events','#resol tau_{1}^{(0.5)}','resol #tau_{1}^{(0.5)}'],['Events','resol #tau_{1}^{(1)}','resol #tau_{1}^{(1)}'],['Events','resol #tau_{1}^{(2)}','resol #tau_{1}^{(2)}'],
    ['Events','#resol tau_{2}^{(0.5)}','resol #tau_{2}^{(0.5)}'],['Events','resol #tau_{2}^{(1)}','resol #tau_{2}^{(1)}'],['Events','resol #tau_{2}^{(2)}','resol #tau_{2}^{(2)}'],
    ['Events','#resol tau_{3}^{(0.5)}','resol #tau_{3}^{(0.5)}'],['Events','resol #tau_{3}^{(1)}','resol #tau_{3}^{(1)}'],['Events','resol #tau_{3}^{(2)}','resol #tau_{3}^{(2)}'],
    ['Events','#resol tau_{4}^{(0.5)}','resol #tau_{4}^{(0.5)}'],['Events','resol #tau_{4}^{(1)}','resol #tau_{4}^{(1)}'],['Events','resol #tau_{4}^{(2)}','resol #tau_{4}^{(2)}'],
    ['Events','resol #tau_{2,1}^{(1)}','resol #tau_{2,1}^{(1)}'],['Events','resol #tau_{3,2}^{(1)}','resol #tau_{3,2}^{(1)}'],
    ])
Range_topSel=[
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.]
    
    
]
print (len(Hists_topSel),len(Histnames_topSel),len(Range_topSel))

14 14 14


In [60]:
print (Histnames_topSel[0][0]
      )

Events


In [61]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [62]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(26)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0], rebin=False))
    TTJets.SetLineColor(ROOT.kRed+5)
    TTJets.SetLineWidth(1)
    TTJets.SetMarkerSize(1)
    TTJets.SetMarkerStyle(26)
    TTJets.SetMarkerColor(ROOT.kRed+5)
    TTJets.SetFillStyle(0)
    
    
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    TTSemileptonic.GetYaxis().SetTitle(Histnames_topSel[i][0])
    TTSemileptonic.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    TTJets.Rebin(2)
    TTSemileptonic.Draw("ep")
    TTJets.Draw("ep samee")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"Powheg+Pythia8","p")
    l[i].AddEntry(TTJets,"aMC@NLO+Pythia8","p")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("topSel/"+Hists_topSel[i]+"_powheg_p8.png")
    c[i].Close()

0 resolJet_tau_0p5_1_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #tau_{1}^{(0.5)}
1 resolJet_tau_1_1_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #tau_{1}^{(1)}
2 resolJet_tau_2_1_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #tau_{1}^{(2)}
3 resolJet_tau_0p5_2_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #tau_{2}^{(0.5)}
4 resolJet_tau_1_2_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #tau_{2}^{(1)}
5 resolJet_tau_2_2_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #tau_{2}^{(2)}
6 resolJet_tau_0p5_3_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #tau_{3}^{(0.5)}
7 resolJet_tau_1_3_topSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 25702.07 365.34 32392787477.90801
resol #ta

Info in <TCanvas::Print>: png file topSel/resolJet_tau_0p5_1_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_1_1_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_2_1_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_0p5_2_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_1_2_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_2_2_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_0p5_3_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_1_3_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_2_3_topSel_powheg_p8.png has been created
Info in <TCanvas::Print>: png file topSel/resolJet_tau_0p5_4_topSel_powheg_p8.png has been created
Info in <TCanvas::Prin

#### 